In [ ]:
import numpy as np
import MarketDataFetcher
import asyncioa
import nest_asyncio
import BinanceBackTest
import utils

nest_asyncio.apply()

In [ ]:
symbols = ['BTCUSDT', 'ADAUSDT', 'XRPUSDT']
intervals = ['1m', '3m', '5m', '1h']

BBT = BinanceBackTest.DataManager(symbols=symbols,
                                  intervals=intervals,
                                  start_date='2024-8-1 00:00:00',
                                  end_date='2024-11-03 23:59:59')
asyncio.run(BBT.read_data_run(download_if_missing=True))

In [ ]:
range_length_data, kline_data, indices_data, closing_sync_data = asyncio.run(BBT.read_data_run(download_if_missing=False))

In [ ]:
dummy = BBT.convert_kline_data_array(kline_data=kline_data)

In [ ]:
symbol = 'BTCUSDT'
target_interval = '3m'

symbols = list(kline_data.keys())
interval = list(kline_data[symbols[0]].keys())

result = {}

# convert_data = self.convert_kline_data_array(kline_data=kline_data)


empty_data = []
for i in dummy[symbol][target_interval]:
    open_timestamp = i[0]
    close_timestamp = i[6]
    condition = np.where((
        dummy[symbol]['1m'][:,0]>=open_timestamp)&
    (dummy[symbol]['1m'][:,6]<=close_timestamp))[0]
    for n in condition:
        if n == condition[-1]:
            empty_data.append(i)
            continue
        if n == condition[0]:
            new_data = [
                open_timestamp,
                dummy[symbol]['1m'][n][1],
                dummy[symbol]['1m'][n][2],
                dummy[symbol]['1m'][n][3],
                dummy[symbol]['1m'][n][4],
                dummy[symbol]['1m'][n][5],
                close_timestamp,
                dummy[symbol]['1m'][n][7],
                dummy[symbol]['1m'][n][8],
                dummy[symbol]['1m'][n][9],
                dummy[symbol]['1m'][n][10],
                0,
            ]
        else:
            new_data = [
                open_timestamp,
                dummy[symbol]['1m'][n][1],
                max(dummy[symbol]['1m'][n][2], empty_data[-1][2]),
                min(dummy[symbol]['1m'][n][3], empty_data[-1][3]),
                dummy[symbol]['1m'][n][4],
                dummy[symbol]['1m'][n][5] + empty_data[-1][5],
                close_timestamp,
                dummy[symbol]['1m'][n][7] + empty_data[-1][7],
                dummy[symbol]['1m'][n][8] + empty_data[-1][8],
                dummy[symbol]['1m'][n][9] + empty_data[-1][9],
                dummy[symbol]['1m'][n][10] + empty_data[-1][10],
                0,
            ]
        empty_data.append(new_data)

In [ ]:
np.array(empty_data)

In [ ]:
import utils

symbol = 'BTCUSDT'
target_interval = '3m'

symbols = list(kline_data.keys())
interval = list(kline_data[symbols[0]].keys())

result = {}

# convert_data = self.convert_kline_data_array(kline_data=kline_data)
obj = utils.DataContainer()

main_data = []
for symbol, kline_data_symbol in kline_data.items():
    a_ = [kline_data[symbol][intervals[0]]]
    for interval, _ in kline_data_symbol.items():
        if interval == intervals[0]:
            continue
        d_ = []
        for i in dummy[symbol][target_interval]:
            open_timestamp = i[0]
            close_timestamp = i[6]
            condition = np.where((
                dummy[symbol]['1m'][:,0]>=open_timestamp)&
            (dummy[symbol]['1m'][:,6]<=close_timestamp))[0]
            for n in condition:
                if n == condition[-1]:
                    empty_data.append(i)
                    continue
                if n == condition[0]:
                    new_data = [
                        open_timestamp,
                        dummy[symbol]['1m'][n][1],
                        dummy[symbol]['1m'][n][2],
                        dummy[symbol]['1m'][n][3],
                        dummy[symbol]['1m'][n][4],
                        dummy[symbol]['1m'][n][5],
                        close_timestamp,
                        dummy[symbol]['1m'][n][7],
                        dummy[symbol]['1m'][n][8],
                        dummy[symbol]['1m'][n][9],
                        dummy[symbol]['1m'][n][10],
                        0,
                    ]
                else:
                    new_data = [
                        open_timestamp,
                        dummy[symbol]['1m'][n][1],
                        max(dummy[symbol]['1m'][n][2], empty_data[-1][2]),
                        min(dummy[symbol]['1m'][n][3], empty_data[-1][3]),
                        dummy[symbol]['1m'][n][4],
                        dummy[symbol]['1m'][n][5] + empty_data[-1][5],
                        close_timestamp,
                        dummy[symbol]['1m'][n][7] + empty_data[-1][7],
                        dummy[symbol]['1m'][n][8] + empty_data[-1][8],
                        dummy[symbol]['1m'][n][9] + empty_data[-1][9],
                        dummy[symbol]['1m'][n][10] + empty_data[-1][10],
                        0,
                    ]
            # obj.set_data(data_name=f'interval_{interval}', data=[new_data])
                d_.append(new_data)
        main_data.append(d_)
    # main_data = np.array(main_data)


In [ ]:
main_data

In [14]:
import numpy as np

# 원본 다차원 리스트
data = [
    [[1, 2, 3], [4, 5, 6]],         # 길이 2
    [[7, 8, 9]],                    # 길이 1
    [[10, 11, 12], [13, 14, 15], [16, 17, 18]]  # 길이 3
]

# 최대 길이 계산 (상위 차원에서 최대 길이 - 1)
max_length = max(len(sublist) for sublist in data)

# 패딩 처리
padded_data = np.array([
    [[0, 0, 0]] * (max_length - len(sublist)) + sublist if len(sublist) < max_length else sublist[:max_length]
    for sublist in data
])

print(padded_data)


[[[ 0  0  0]
  [ 1  2  3]
  [ 4  5  6]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 7  8  9]]

 [[10 11 12]
  [13 14 15]
  [16 17 18]]]


In [ ]:
path = os.path.join(os.path.dirname(os.getcwd()), 'DataStore/kline_data.json')

kline_data = utils._load_json(path)

def convert_to_container(kline_data):
    container_data = utils.DataContainer()
    
    map_symbol = {}
    map_interval = {}
    
    symbols = list(kline_data.keys())
    intervals = list(kline_data[symbols[0]].keys())
    
    for idx_i, interval in enumerate(intervals):
        map_interval[interval] = idx_i
    
        dummy_data = []
        
        for idx_s, symbol in enumerate(symbols):
            map_symbol[symbol] = idx_s
    
            target_data = kline_data[symbol][interval]
            dummy_data.append(target_data)
    
        dummy_data = np.array(dummy_data)
        container_data.set_data(data_name = f'interval_{interval}', data=dummy_data)
    return map_symbol, map_interval, container_data